# LIVRIA : Recommandation des thèmes et des livres 

C'est dans ce notebook -que l'on peut considérer comme le dernier - que mettons en lumières les recommendations de nos différents modèles

### Import des données en background

In [1]:
from ui import *
livria = Livria()

# Prédiction des thèmes

In [2]:
display_quest()
# Veuillez répondre aux questions qui suivent :

RadioButtons(description='Renseignez votre sexe', options=('Homme', 'Femme'), value='Homme')

SelectMultiple(description='Sélectionnez les items qui vous définissent le mieux', options=('Calme', 'Intellec…

SelectMultiple(description='Quels sont vos passe-temps favoris ?', options=('Sport', 'Dessin', 'Rien faire', '…

SelectMultiple(description="Qu'est ce qui vous plaît dans un livre ?", options=('Voyage', 'FacileLire', 'Refle…

In [3]:
livria.enr_crits(s_sexe.value, s_pers.value, s_passe_t.value, s_attentes.value)
livria.car_input

,Agite,Altruiste,Ambitieux,Amusant,Autoritaire,Aventurier,Calme,Connaissance,Consciencieux,Creatif,Cuisine,Curieux,Dessin,Esprit,FacileLire,Geek,Intellectuel,Introverti,Jaloux,Jeux videos,Meditation,Pantouflard,Personnage,Reflechir,Reserve,Rien faire,Sexe,Sociable,Sport,Sportif,Style,Theatre,Tout,Voyage
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [4]:
livria.predict()

**Calculs des prédictions pour ArtsCulture...**
[0]
**Calculs des prédictions pour BdComics...**
[0]
**Calculs des prédictions pour DocMedia...**
[0]
**Calculs des prédictions pour Erotisme...**
[0]
**Calculs des prédictions pour Esoterisme...**
[0]
**Calculs des prédictions pour HistGeo...**
[0]
**Calculs des prédictions pour Jeunesse...**
[0]
**Calculs des prédictions pour LittEtrangere...**
[0]
**Calculs des prédictions pour LoisirVie...**
[0]
**Calculs des prédictions pour Philosophie...**
[0]
**Calculs des prédictions pour RomanFiction...**
[0]
**Calculs des prédictions pour SHS...**
[0]
**Calculs des prédictions pour SanteBE...**
[0]
**Calculs des prédictions pour ScienceTechnique...**
[0]


,ArtsCulture,BdComics,DocMedia,Erotisme,Esoterisme,HistGeo,Jeunesse,LittEtrangere,LoisirVie,Philosophie,RomanFiction,SHS,SanteBE,ScienceTechnique
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
livria.rename()
livria.themes_output

,art,comics,movies,erotica,mystery,historical,childhood,literature,travel,philosophy,fiction,sociology,personal-development,science
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
livria.list_themes()

### Affichage des livres

Ici, nous allons crée la fonction permettant d'afficher proprement les livres avec leur page de couverture, le titre l'auteur et la note moyenne. On commence par nettoyer le DataFrame des colonnes d'ont nous n'avons pas besoin ici.

On crée la fonction qui nous permettra d'afficher les livres :

In [7]:
# On teste la fonction
livresMontres = livres.loc[livres['tag_name'].isin(livria.themes_output)]
show_books(livresMontres) # Elle prend en entrée un dataFrame avec les livres que nous souhaitons afficher

                   Liste des livres qui pourraient vous plaire



## Préparation et entrainement des données pour la prédiction de thèmes

# Recommandations

## A l'entrée de l'application 

Veuillez répondre au questions ci-dessous :

In [8]:
questions()

NameError: name 'questions' is not defined

In [ ]:
def get_recommendations(name,sim):
   # indx=df_book_new.loc[df_book_new['title']==name].index
    indx=book_indices[name]
    sim_scores=list(enumerate(sim[indx]))
    new=sorted(sim_scores,key=lambda x: x[1],reverse=True)
   
    new=new[1:11]
    #print(new)
    book_idx=[x[0] for x in new]
    return (df_book_new['title'].iloc[book_idx])

In [ ]:
sel_m_attentes.value.info()

In [ ]:
book = books.query("book_id == 11")
Image(url= book['image_url'], width=100, height=100)

In [ ]:
total = 0

def doA():
    # not accessing global total
    total = 10

def doB():
    global total
    total = total + 1

def checkTotal():
    # global total - not required as global is required
    # only for assignment - thanks for comment Greg
    print (total)

In [ ]:
doA()
doB()
checkTotal()

In [ ]:
def recommendation(user_id):
    user = book.copy()
    already_read = books_data[books_data['user_id'] == user_id]['book_id'].unique()
    user = user.reset_index()
    user = user[~user['book_id'].isin(already_read)]
    user['Estimate_Score']=user['book_id'].apply(lambda x: algo.predict(user_id, x).est)
    user = user.drop('book_id', axis = 1)
    user = user.sort_values('Estimate_Score', ascending=False)
    print(user.head(10))